In [ ]:
import xarray as xr
import pandas as pd
import rioxarray
from ndpyramid import pyramid_reproject
from carbonplan_data.utils import set_zarr_encoding
from carbonplan_data.metadata import get_cf_global_attrs
import numpy as np

In [ ]:
save_path = "s3://carbonplan-climatetrace/v1.2/map/"

In [ ]:
path = "s3://carbonplan-climatetrace/v1.2/results/global/3000m/raster_split.zarr"
v1_emissions = xr.open_zarr(path)

In [ ]:
biomass = xr.open_zarr("s3://carbonplan-climatetrace/v1.2/results/global/3000m/raster_biomass.zarr")
biomass = biomass.rename({"time": "year"})
biomass = biomass.assign_coords(
    {"year": np.arange(2014, 2021), "lat": v1_emissions.lat, "lon": v1_emissions.lon}
)

In [ ]:
v0_emissions = xr.open_zarr("s3://carbonplan-climatetrace/v0.4/global/3000m/raster_tot.zarr").sel(
    year=slice(2015, 2020)
)

In [ ]:
v0_emissions = v0_emissions.assign_coords(v1_emissions.coords)

In [ ]:
# hack to get hansen tile area
valid_mask = v0_emissions.emissions.sel(year=2018) > -1

In [ ]:
v1_emissions = v1_emissions.fillna(0).where(valid_mask)
biomass = biomass.fillna(0).where(valid_mask)

In [ ]:
full_ds = xr.Dataset(
    {
        "biomass": biomass["AGB"],
        "biomass-na-filled": biomass["AGB_na_filled"],
        "emissions-v1": v1_emissions["emissions_from_clearing"]
        + v1_emissions["emissions_from_fire"],
        "sinks-v1": v1_emissions["sinks"],
        "net-v1": v1_emissions["sinks"]
        + v1_emissions["emissions_from_clearing"]
        + v1_emissions["emissions_from_fire"],
        "emissions-v0": v0_emissions.sel(year=slice(2015, 2020))["emissions"],
    }
)

In [ ]:
full_ds = xr.Dataset({"variable": full_ds.to_array("band")}).rio.write_crs("EPSG:4326")
full_ds = full_ds.rename({"lat": "y", "lon": "x"})

In [ ]:
for dim in full_ds.dims.keys():
    if dim != "band":
        full_ds[dim] = full_ds[dim].astype("float32")
for var in full_ds.data_vars:
    full_ds[var] = full_ds[var].astype("float32")

In [ ]:
# create the pyramid
dt = pyramid_reproject(full_ds, levels=6, extra_dim="band")

In [ ]:
for child in dt.children:
    child.ds = child.ds.where(child.ds < 2e38)
    child.ds = set_zarr_encoding(
        child.ds, codec_config={"id": "zlib", "level": 1}, float_dtype="float32"
    )
    child.ds = child.ds.chunk({"x": 128, "y": 128, "band": -1, "year": -1})
    child.ds["variable"].attrs.clear()
dt.attrs = get_cf_global_attrs()

# write the pyramid to zarr
dt.to_zarr(save_path + "forest_carbon_web_data_v3.pyr", consolidated=True)